In [ ]:
def network_layout(df, net_iteration, iteration, df_out_net=None, option='close'):
        '''
    Creates a table with information for each path in network.
        
    Parameters
    ----------
    
    df: dataframe
         list of locations, group membership and parameters used to generate network
        
    net_iteration: list of list
        a list containing one or several lists, one for each group network that resulted from an single network iteration
    
    iteration_id: int
        iteration identifier
    
    df_out_net: dataframe, optional
        table containing the identifiers of the origin and destination of each path plus the iteration identifier
    
    option: string
        type of network that is generated. There are three possible types:
           - *close* (*default*).
           - *cummulative*.
           - *all*.
    
    Returns
    -------
    
    df_out_net: dataframe, optional
       
    Notes
    -----
    
    This function takes a dataframe with locations, a list of lists containing an ordering of these locations obtained after running 
    *create_network_generator()* function and a iteration identifier number. It generates, or updates, a dataframe with the 
    identifiers of the origin and destination of each path that make up the path network for this specific iteration. 
    
    *Mixing*. This refers to whether only paths between locations within the same group are allowed or not. If mixing occurs,
    a path will be generated that connects locations that belong to two distinct groups. More specifically, between a the first
    location in a group and the last location in the *previous* group. The following observations need to be kept in mind with
    regards to mixing:
    - All pts in a group must have the same 'mix' code.
    - The value of the 'mix' column for all members of the first group is always set to 0 (i.e. no mixing).
    - We identify two degenerative cases if the number of points in a group is less than 2:
      - if the group is the first one to be processed the point will be set as an origin and will be mixed with the points of the
      following group.
      - if not the first group, it will be set as a destination and will be mixed with the points of the previous group.
    
    This function calls __paths_table() function.
    '''

    if df_out_net is None:
        net = {'origin': 'int32', 'destination': 'int32', 'iteration': 'int32'}
        df_out_net= pd.DataFrame(columns=list(net.keys())).astype(net)
        
    prev_pt_id = 0                                                                      # indentifies last point used in
                                                                                        # previous group. Only used for
                                                                                        # path between points in different
                                                                                        # groups.

    groups = df['group'].unique()                                                       # distinct groups in df

    for pos, g in enumerate(groups):                                                    # Loop thru groups

        ids = list(net_iteration[pos])                                                  # create list with pts id in group.
        num_pts_in_grp = len(ids)                                                       # number of pts in group.

        # degenerative cases
        if (num_pts_in_grp == 1) and (pos == 0):                                        # one point in first group only.

            prev_pt_id = ids[0]                                                         # set point to origin of next path.
            sel_next = df['group'] == groups[1]                                         # force second group into mixing
            df.at[sel_next, 'mix'] = 1

        elif (num_pts_in_grp == 1) and (pos > 0):                                       # group with only one point.
            
            prev_pt_id = ids[0]                                                         # set pt to next previous_pt_id.
            ids.insert(0, prev_pt_id)                                                   # previous pt id is path origin.
            df_out_net = __paths_table(ids, iteration, df_out_net, option)
            
        # non-degenerative cases
        else:

            if df.loc[df['group'] == g, 'mix'].iloc[0] == 0:                            # no mixing.
                
                prev_pt_id = ids[-1]
                df_out_net = __paths_table(ids, iteration, df_out_net, option)

            else:                                                                       # mixing.
                ids.insert(0, prev_pt_id)
                prev_pt_id = ids[-1]
                df_out_net = __paths_table(ids, iteration, df_out_net, option)

    return df_out_net

# Tutorial 2. Network Generation, part 2

In this tutorial we explore the different types of network orderings that ```network_layout()``` function from the *netgen* module can generate.

### Imports

In [1]:
import geopandas as gpd
import pandas as pd
import netsim.netgen as ng

### Read a sample shapefile

In [2]:
fn = r'..\data\sample\sample5.shp'

We use geopandas to read the shapefile that contains the location and the columns needed to run the simulation. In case the location file is a simple a text file, e.g. *comma-delimited* or *csv*, use
```python
import pandas as pd
df = pd.read_csv(filename)
```
instead.


In [3]:
df= gpd.read_file(fn)
df

,id,seq,group,mix,easting,northing,geometry
0,0,1,1,0,530782,4389390,POINT (530782 4389390)
1,1,1,1,0,531119,4388860,POINT (531119 4388860)
2,2,1,1,0,530403,4388580,POINT (530403 4388580)
3,3,1,1,0,530503,4388620,POINT (530503 4388620)
4,4,1,1,0,530729,4388930,POINT (530729 4388930)
5,5,1,1,0,530606,4389150,POINT (530606 4389150)


Let's change the number of groups so that we end up with two groups, of two and four locations.

In [4]:
sel = df['id'].isin([2, 3, 4, 5])
df.loc[sel,'group']= 2
df

,id,seq,group,mix,easting,northing,geometry
0,0,1,1,0,530782,4389390,POINT (530782 4389390)
1,1,1,1,0,531119,4388860,POINT (531119 4388860)
2,2,1,2,0,530403,4388580,POINT (530403 4388580)
3,3,1,2,0,530503,4388620,POINT (530503 4388620)
4,4,1,2,0,530729,4388930,POINT (530729 4388930)
5,5,1,2,0,530606,4389150,POINT (530606 4389150)


We run the ```setup()``` function in the *netgen* module to check whether the table with the information needed to run the simulation has all the appropriate columns and values within these. 

In [5]:
df = ng.setup(df)


 No corrections or errors !! 


### Creating a network generator

The next step we run the ```create_network_generator()``` function to create a *network generator* that we can  use to produce different versions, or iterations, of our network.

In [6]:
netgentor, net_info, total_iterations = ng.create_network_generator(df)


 iteration broken per group....

   group    iter_type  num_iter  num_loc
0      1  permutation         2        2
1      2  permutation        24        4

 total number of iterations.... 48


### Creating a network of paths

 The ```network_layout()``` function transforms an iteration generated by the generator created using the  ```create_network_generator()``` function into a network of paths.

In [7]:
sample_iteration = list(next(netgentor))
sample_iteration

[(0, 1), (2, 3, 4, 5)]

Let's generate the a 'close' network of path, this essentially creates a close circuit of paths amongst the different locations in a group.

In [9]:
df_net = ng.network_layout(df, sample_iteration, 1)
df_net

,origin,destination,iteration
0,0,1,1
1,1,0,1
2,2,3,1
3,3,4,1
4,4,5,1
5,5,2,1


Here are a couple of things to note. First, by default ```network_layout()``` will generate a **close** network (*acyclic* network) amongst all the locations within a group (unless we change the 'mix' column, *see below*) .   Notice that the first loop just connects two locations 0 to 1 and 1 to 0 (this only make sense if the paths we are generating are based on an *anysotropic* algorithm). The second loop conects 2 to 3, 3 to 4, 4 to 5 and 5 to 2.

Let's see what other networks we can generate,

In [12]:
df_net = ng.network_layout(df, sample_iteration, 1, option='cummulative')
df_net

,origin,destination,iteration
0,0,1,1
1,2,3,1
2,4,2,1
3,4,3,1
4,5,2,1
5,5,3,1
6,5,4,1


With the 'cummulative' option, locations **in each group** are connected in the following way, the first path in the network is defined using the first two location ids of the iteration. The remaining location ids are used as origins of the subsequent paths such that destinations are *all* location ids *previous* to the current origin id. In this particular case, we only got one path for the first group as there are only two locations. For the second group, the first path is defined between locations 2 and 3 then paths are defined from location 4 to locations 2 and 3 and lastly, paths are defined from location 5 to all of the previous locations, so locations 2, 3 and 4. The last type of network is 'all' where all locations in a group are connected to all of the other locations,

In [13]:
df_net = ng.network_layout(df, sample_iteration, 1, option='all')
df_net

,origin,destination,iteration
0,0,1,1
1,1,0,1
2,2,3,1
3,2,4,1
4,2,5,1
5,3,2,1
6,3,4,1
7,3,5,1
8,4,2,1
9,4,3,1


The output of all of these network can change slighlty if we change the value of the 'mix' column for a group. Currently, our group 1 has a 'mix' value of '0' so this means that it will not mix with (the first element of) group 2. Let's change it to '1' and see what happens.  

In [29]:
df.loc[df['group']==1,'mix'] = 1

In [34]:
ng.setup(df)


 No corrections or errors !! 


,id,seq,group,mix,easting,northing,geometry
0,0,1,1,0,530782,4389390,POINT (530782 4389390)
1,1,1,1,0,531119,4388860,POINT (531119 4388860)
2,2,1,2,1,530403,4388580,POINT (530403 4388580)
3,3,1,2,1,530503,4388620,POINT (530503 4388620)
4,4,1,2,1,530729,4388930,POINT (530729 4388930)
5,5,1,2,1,530606,4389150,POINT (530606 4389150)


In [30]:
df.loc[df['group']==2,'mix'] = 1
df

,id,seq,group,mix,easting,northing,geometry
0,0,1,1,0,530782,4389390,POINT (530782 4389390)
1,1,1,1,0,531119,4388860,POINT (531119 4388860)
2,2,1,2,1,530403,4388580,POINT (530403 4388580)
3,3,1,2,1,530503,4388620,POINT (530503 4388620)
4,4,1,2,1,530729,4388930,POINT (530729 4388930)
5,5,1,2,1,530606,4389150,POINT (530606 4389150)


In [33]:
df_net= ng.network_layout(df, sample_iteration, 1, option='all')
df_net

,origin,destination,iteration
0,0,1,1
1,1,0,1
2,1,2,1
3,1,3,1
4,1,4,1
5,1,5,1
6,2,1,1
7,2,3,1
8,2,4,1
9,2,5,1


Next, given a single network iteration, we call the ```network_layout()``` function, that uses the **mix** parameter to determine the ordering of paths. This function expects the *corrected* original dataframe (containing all locations that make-up the network), an outputs a network dataframe with 'origin', destination' and 'iteration' fields, and the iteration number. It will return the output network dataframe with the origin-destination and iteration number for each path in the network. 

In [56]:
for i in range(num_iterations):
    sample_iteration = list(next(netgentor))
    df_out_net = network_layout(df, sample_iteration, i, df_out_net, option='close')

In [146]:
gmax= 4
gt = 0
T = 12

In [147]:
for i in range(100):
    gt= gt - gt/T + (1 - gt/gmax)
    print(gt)

1.0
1.6666666666666665
2.111111111111111
2.4074074074074074
2.6049382716049383
2.736625514403292
2.824417009602195
2.88294467306813
2.9219631153787535
2.947975410252502
2.965316940168335
2.9768779601122235
2.984585306741482
2.989723537827655
2.993149025218437
2.995432683478958
2.996955122319305
2.9979700815462036
2.9986467210308025
2.9990978140205353
2.999398542680357
2.9995990284535714
2.9997326856357143
2.99982179042381
2.999881193615873
2.9999207957439156
2.9999471971626104
2.9999647981084068
2.9999765320722713
2.9999843547148477
2.9999895698098986
2.9999930465399327
2.9999953643599553
2.9999969095733032
2.9999979397155356
2.9999986264770238
2.9999990843180155
2.9999993895453434
2.9999995930302292
2.999999728686819
2.999999819124546
2.9999998794163636
2.999999919610909
2.999999946407273
2.999999964271515
2.99999997618101
2.9999999841206733
2.999999989413782
2.9999999929425214
2.9999999952950143
2.999999996863343
2.9999999979088954
2.9999999986059303
2.99999999907062
2.99999999938041

In [101]:
for i in range(10):
    gt= gt_1 - gt_1/T + (1 - gt_1/gmax)
    print(gt)
    gt_1 = gt

-124923.47242103516
-206124.529494708
-340106.2736662682
-561176.1515493426
-925941.4500564152
-1527804.192593085
-2520877.7177785905
-4159449.0343346745
-6863091.706652213
-11324102.115976151
-18684769.291360646
-30829870.13074507
-50869286.51572937
-83934323.55095346
-138491634.6590732
-228511197.98747078
-377043477.4793268
-622121738.6408893
-1026500869.5574672
-1693726435.569821
